## 2 of 2: Drugs of abuse in biological samples: Casework Sample Data Analysis

This is notebook 2 of 2 and is designed to import two data exports from Masshunter which contains data for the analysis of drugs in blood samples - one contains quality control data and the other contains casework sample data. The QC data is assessed for accuracy against acceptable limits and casework samples are processed for the presence of any drug compounds of interest. Any quality failures (qualifier ratios, low ISTD response or replicate disagreement) are analysed and presented where needed.
(for future revisions - only require 1 Masshunter export)

In [27]:
import pandas as pd
import numpy as np

Load the Quality Control results

In [28]:
df_cw = pd.read_excel('Cases.xlsx')

In [29]:
cw_cols = []
cw_cols.append(df_cw.columns)

to_append = df_cw.iloc[0,:].tolist()

i = 0
for item in df_cw.columns:
    df_cw.rename(columns={item: item + ' ' + to_append[i]}, inplace=True)
    i+=1

df_cw.drop([0], inplace=True)
df_cw.columns = df_cw.columns.str.lower().str.replace(' ', '_').str.replace('unnamed:_', '').str.replace('.', '').str.replace('-', '_').str.replace('(', '').str.replace(')', '').str.replace('1_d', 'd')

In [30]:
## Clean up the columns
# dev = ISTD response % deviation (ideal value = 0)
# rr = response ratio (drug area / istd area)
# rrt = relative retention time (retention time of drug / retention time of istd)

col_list = df_cw.columns.tolist()
num = 1
num2 = 1
istd_text = 'istd'
for i in range(0, len(col_list)):
    if 'cocaine' in col_list[i]: drug = 'cocaine'
    if 'benzoylecgonine' in col_list[i]: drug = 'bze'
    if 'mdma' in col_list[i]: drug = 'mdma'
    if 'mda' in col_list[i]: drug = 'mda'
    if 'cocaethylene' in col_list[i]: drug = 'ce'
    if '_d' in col_list[i]: drug = drug + '_istd'
        
    if 'qualifier' in col_list[i]:
        if 'istd' in drug: 
            col_list[i] = drug + '_qr'
            continue
        
    if '_dev' in col_list[i]:
        if 'istd' in drug: 
            col_list[i] = drug + '_dev'
            continue
    
    
    if '_rr' in col_list[i]:
        if num == 1: 
            col_list[i] = drug + '_rr'
            num += 1
        elif not num ==1: 
            col_list[i] = drug + '_rrt'
            num = 1
            
    if 'qualifier' in col_list[i]:
        if num2 == 1: 
            col_list[i] = drug + '_qr_' + str(num2)
            num2 += 1
        elif not num2 ==1: 
            col_list[i] = drug + '_qr_' + str(num2)
            num2 = 1
            
df_cw.columns = col_list

In [12]:
df_cw.columns = df_cw.columns.str.replace('_calc', '').str.replace('_results', '')
df_cw.columns = df_cw.columns.str.replace('_(istd)_results_istd_resp_%_dev', '_dev').str.replace('_results', '').str.replace('benzoylecgonine', 'bze').str.replace('cocaethylene', 'ce')

In [13]:
# round all numbers to appropriate decimal places

for col in df_cw.columns:
    if type(df_cw[col][1]) == float:
        if 'conc' in col or '_dev' in col or 'qr' in col: df_cw[col] = df_cw[col].apply(lambda x: round(x, 2))
        if 'rr' in col: df_cw[col] = df_cw[col].apply(lambda x: round(x, 4))

In [14]:
df_cw.set_index('sample_name', inplace=True)

In [15]:
df_cals = df_cw[df_cw.index.str.startswith('Blood CAL')]

In [16]:
# dictionaries of target values for drug parameters

comps = ['cocaine', 'bze', 'mda', 'mdma', 'ce']
rows = ['_conc', '_rr', '_rrt', '_qr_1', '_qr_2', '_istd_dev', '_istd_qr']
toxicity = {'mda': 50, 'mdma': 100, 'ce': 200, 'cocaine': 300, 'bze': 500}

cals_qr_1 = {}
cals_qr_2 = {}
cals_rrt = {}

for comp in comps:
    cals_qr_1.update({comp: round(df_cals[comp + '_qr_1'].mean(),2)})
    cals_qr_2.update({comp: round(df_cals[comp + '_qr_2'].mean(),2)})
    cals_rrt.update({comp: round(df_cals[comp + '_rrt'].mean(),4)})

In [17]:
# Make dataframe of all cases
df_all_cases = pd.DataFrame()

for case in df_cw.index:
    if 'Case' in case:
        df_all_cases = df_all_cases.append(df_cw.loc[case])
num_cases = int(len(df_all_cases) / 2)

# FUNCTIONS FOR CASEWORK ASSESSMENT



In [18]:
# Check qualifier ratios for the case are within tolerance based on average of calibration standards

def assess_qualifiers(comp, qr1, qr2):
    
    rat1 = cals_qr_1[comp]
    rat2 = cals_qr_2[comp]    

    if qr1 < rat1 * 0.8 or qr1 > rat1 * 1.2 or qr2 < rat2 * 0.8 or qr2 > rat2 * 1.2: return False
    else: return True

In [19]:
# Check relative retention time of case against average of calibration standards

def assess_rrt(comp, rrt):
    cal_rrt = cals_rrt[comp]
    
    if rrt < cal_rrt * 0.9 or rrt > cal_rrt * 1.1: return False
    else: return True

In [20]:
# If either of the 'check' functions returns False then print a cautionary note

def caution_stat(comp, qr1, qr2, rrt):
    if assess_qualifiers(comp, qr1, qr2) == False:
        print('One or more qualiying transitions failed')
    if assess_rrt(comp, rrt) == False:
        print('Relative retention time failed: check chromatograms for interference')
    else: print('Quality Control cautions: none')

In [21]:
# Check concs > cal1 (any compound detected < cal1 is not reported as it is not toxicologically relevant)
# If > cal1 then check 'assessment' functions for quality control of the sample

def assess_conc(sample_a, sample_b, comp, conc1, conc2, cal1):

    ''' Determine if any concentration measured for both samples aliquots is > reporting limit. 
        Determine is the replicate agreement is acceptable. Return the results and interpret 
        for toxicological significance'''
    
    avg = round((conc1 + conc2)/2,2)
    reps_ok = False
    
    if avg * 0.8 < conc1 or avg * 0.8 < conc2: reps_ok = True
   
    if conc1 < cal1 or conc2 < cal1:
        print('{} N.D. \n'.format(comp.capitalize()))
    if avg > cal1:
        print('Sample A - {} concentration: {} ng/mL'.format(comp.capitalize(), conc1))
        print('Sample B - {} concentration: {} ng/mL'.format(comp.capitalize(), conc2))
        print('Average {} concentration: {} ng/mL'.format(comp, avg))
        
        caution_stat(comp, df_all_cases[comp + '_qr_1'][sample_a], df_all_cases[comp + '_qr_2'][sample_a], df_all_cases[comp + '_rrt'][sample_a])

        if not reps_ok: print('Poor replicate agreement')
        if avg < toxicity[comp]: 
            print('Blood {} concentration is non-toxic \n'.format(comp))
        else: print('Toxic blood {} concentration detected \n'.format(comp))

In [22]:
# Output the results for all cases

case = 1
for i in range(num_cases):

    sample_a_loc = (case * 2) -2
    sample_b_loc = (case * 2) -1
    sample_a = df_all_cases.iloc[sample_a_loc]
    sample_b = df_all_cases.iloc[sample_b_loc]
    case += 1
    print(df_all_cases.index[sample_a_loc] + ' / ' + df_all_cases.index[sample_b_loc])
    
    for comp in comps:
        assess_conc(
            df_all_cases.index[sample_a_loc], 
            df_all_cases.index[sample_b_loc], 
            comp, 
            sample_a[comp + '_conc'], 
            sample_b[comp + '_conc'], 
            df_cals[comp+'_conc']['Blood CAL 1'])
    print('\n \n')

Case-ME-83338856 / Case-ME-72783323
Cocaine N.D. 

Bze N.D. 

Sample A - Mda concentration: 26.28 ng/mL
Sample B - Mda concentration: 27.26 ng/mL
Average mda concentration: 26.77 ng/mL
Quality Control cautions: none
Blood mda concentration is non-toxic 

Mdma N.D. 

Ce N.D. 


 

Case-ME-86069787 / Case-ME-97195639
Cocaine N.D. 

Bze N.D. 

Mda N.D. 

Mdma N.D. 

Ce N.D. 


 

Case-ME-36609722 / Case-ME-97367914
Cocaine N.D. 

Bze N.D. 

Mda N.D. 

Mdma N.D. 

Ce N.D. 


 

Case-ME-14210839 / Case-ME-67396097
Cocaine N.D. 

Bze N.D. 

Mda N.D. 

Mdma N.D. 

Ce N.D. 


 

Case-ME-71410857 / Case-ME-86429868
Cocaine N.D. 

Bze N.D. 

Sample A - Mda concentration: 164.37 ng/mL
Sample B - Mda concentration: 163.15 ng/mL
Average mda concentration: 163.76 ng/mL
Quality Control cautions: none
Toxic blood mda concentration detected 

Mdma N.D. 

Ce N.D. 


 

Case-ME-61903895 / Case-ME-50471275
Cocaine N.D. 

Bze N.D. 

Mda N.D. 

Mdma N.D. 

Ce N.D. 


 

Case-ME-83109483 / Case-ME-58495501
C